# Evaluating the Deep Neural Network Model for Food ordering smart assistant.

### Importing the libraries

In [1]:
import nltk
import numpy
import tflearn 
import tensorflow
import json
import pickle

from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


### Importing the dataset

In [2]:
with open('data.json') as file:
    data = json.load(file)
    

### Function to save an object using file

In [3]:
def save(dObj, sFilename):
      '''Given an object and a file name, write the object to the file using pickle.'''

      f = open(sFilename, "wb")
      p = pickle.Pickler(f)
      p.dump(dObj)
      f.close()


### Function to Preprocess text 

In [4]:
def preprocess(text):
    ''' Accepts text as a parameter and performs the following preprocessing: 1. Tokenization
    2. Lemmatization 3. Convert words to lower case. Returns preprocessed tokens as output.
    Stop words and punctuations provide valuable context and hence are not preprocessed.'''

    tokens = []
    words = nltk.word_tokenize(text)

    for word in words:
        tokens.append(lemmatizer.lemmatize(word.lower()))

    return tokens
    

### Preprocess the data

In [5]:
tokens = set()
labels = set()

document_x = []
document_y = []

for intent in data['intents']:
    for question in intent['questions']:
        words = preprocess(question)
        tokens.update(words)
        document_x.append(words)
        document_y.append(intent["label"]) 
        labels.add(intent['label'])


### Using One-Hot Encoding to create bag of words

In [6]:
X = []
y = []

tokens = list(tokens)
labels = list(labels)

class_zeros = [0 for _ in range(len(labels))]

for i, document in enumerate(document_x):
    bag = []
    class_row = class_zeros[:]

    for token in tokens:
        if token in document:
            bag.append(1)
        else:
            bag.append(0)

    
    class_row[labels.index(document_y[i])] = 1
    X.append(bag)
    y.append(class_row)


### Defining the DNN with 2 hidden layers with each having 8 fully connected neurons and a output layer with softmax activation function

In [7]:
ops.reset_default_graph()

net = tflearn.layers.core.input_data(shape=[None, len(X[0])])
net = tflearn.layers.core.fully_connected(net, 8)
net = tflearn.layers.core.fully_connected(net, 8)
net = tflearn.layers.core.fully_connected(net, len(y[0]), activation="softmax")
net = tflearn.layers.estimator.regression(net)

model = tflearn.DNN(net)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Evaluating the model using training and testing set (80% training and 20% testing split)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train = numpy.array(X_train)
y_train = numpy.array(y_train)

model.fit(X_train, y_train, n_epoch=1000, batch_size=8, show_metric=True)

X_test = numpy.array(X_test)
y_test = numpy.array(y_test)

score = model.evaluate(X_test, y_test)
print('Test accuracy: %0.4f%%' % (score[0] * 100))


Training Step: 12999  | total loss: 0.02052 | time: 0.032s
| Adam | epoch: 1000 | loss: 0.02052 - acc: 0.9998 -- iter: 096/101
Training Step: 13000  | total loss: 0.02085 | time: 0.035s
| Adam | epoch: 1000 | loss: 0.02085 - acc: 0.9998 -- iter: 101/101
--
Test accuracy: 80.7692%
